![image](https://drive.google.com/u/0/uc?id=15DUc09hFGqR8qcpYiN1OajRNaASmiL6d&export=download)

# **Taller No. 11 - ISIS4825**

## **Perceptron Multicapa y Redes Neuronales Profundas**
1. [**Objetivos**](#id1)
2. [**Problema**](#id2)
3. [**Importando las librerías necesarias para el laboratorio**](#id3)
4. [**Visualización y Análisis Exploratorio**](#id4)
5. [**Preparación de los Datos**](#id5)
6. [**Modelamiento**](#id6)
7. [**Predicción**](#id7)
8. [**Validación**](#id8)
9. [**Trabajo Asíncrono**](#id9)

## **Objetivos**<a name="id1"></a>
- Introducirnos a las redes neuronales a través del perceptron multicapa.
- Ver algunas funciones básicas de `Keras` y de `TensorFlow`.

## **Problema**<a name="id2"></a>
- La idea del siguiente laboratorio es clasificar imágenes de escritura de dígitos desde el 0 hasta el 9.

## **Notebook Configuration**

In [1]:
!shred -u setup_colab_general.py
!wget -q "https://github.com/jpcano1/python_utils/raw/main/setup_colab_general.py" -O setup_colab_general.py
import setup_colab_general as setup_general
setup_general.setup_general()

shred: setup_colab_general.py: failed to open for writing: No such file or directory



General Functions Enabled Successfully


## **Importando las librerías necesarias para el laboratorio**<a name="id3"></a>

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
plt.style.use("seaborn-dark")
import seaborn as sns

from utils import general as gen

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, confusion_matrix

### **Carga de Datos**

In [ ]:
(full_X_train, full_y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
full_X_train.shape, X_test.shape

## **Visualización y Análisis Exploratorio**<a name="id4"></a>

In [ ]:
np.random.seed(0)
random_sample = np.random.choice(len(full_X_train), size=9)

In [ ]:
gen.visualize_subplot(full_X_train[random_sample], 
                      full_y_train[random_sample], 
                      (3, 3), (6, 6))

In [ ]:
dist = np.unique(full_y_train, return_counts=True)

In [ ]:
sns.barplot(x=dist[0], y=dist[1])
plt.show()

In [ ]:
dist[1].mean()

## **Preparación de los Datos**<a name="id5"></a>

### **Normalización**

In [ ]:
full_X_train, X_test = full_X_train / 255., X_test / 255.

In [ ]:
full_X_train.max()

In [ ]:
X_test.max()

In [ ]:
full_X_train.mean()

### **Train Set, Validation Set, Test Set**

In [ ]:
sss = StratifiedShuffleSplit(test_size=0.2, random_state=1234)

In [ ]:
for train_index, val_index in sss.split(full_X_train, full_y_train):
    X_train, y_train = full_X_train[train_index], full_y_train[train_index]
    X_val, y_val = full_X_train[val_index], full_y_train[val_index]

## **Modelamiento**<a name="id6"></a>

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=(28, 28)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dense(10, activation="softmax", kernel_initializer="he_normal")
])

In [ ]:
optimizer = keras.optimizers.Adam(lr=5e-4)
metrics = [keras.metrics.SparseCategoricalAccuracy(name="acc")]
loss = keras.losses.SparseCategoricalCrossentropy(name="loss")

In [ ]:
model.compile(optimizer=optimizer, metrics=metrics, loss=loss)

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, 
                    validation_data=(X_val, y_val), 
                    epochs=1)

## **Predicción**<a name="id7"></a>

In [ ]:
np.random.seed(1234)
random_sample = np.random.choice(len(X_test), 9)

In [ ]:
y_pred = model.predict(X_test[random_sample])

In [ ]:
gen.visualize_subplot(X_test[random_sample], 
                      y_pred.argmax(axis=1), 
                      (3, 3), (6, 6))

## **Validación**<a name="id8"></a>

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=1)

In [ ]:
loss

In [ ]:
acc

In [ ]:
plt.plot(history.history["acc"], label="Train Acc")
plt.plot(history.history["val_acc"], Label="Val Acc")
plt.xlabel("Epoch")
plt.ylabel("Acc")
plt.legend(loc="best")
plt.show()

In [ ]:
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="best")
plt.show()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred.argmax(axis=1))

In [ ]:
plt.matshow(conf_matrix, cmap="gray")
plt.title("Confusion Matrix")
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.show()

In [ ]:
norm_conf_mat = conf_matrix / conf_matrix.sum(axis=1, keepdims=True)
np.fill_diagonal(norm_conf_mat, 0)

In [ ]:
plt.matshow(norm_conf_mat, cmap="gray")
plt.title("Confusion Matrix")
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.show()

In [ ]:
precision_score(y_test, y_pred.argmax(axis=1), average="weighted")

In [ ]:
recall_score(y_test, y_pred.argmax(axis=1), average="weighted")

## **Trabajo Asíncrono**<a name="id9"></a>
1. Entrenar esta misma red sobre el dataset de CIFAR-10 utilizando una [GPU](https://www.tensorflow.org/guide/gpu). Para cargar CIFAR-10, lo podrá hacer con la siguiente línea de código: `keras.datasets.cifar10.load_data()`.
2. Responder las siguientes preguntas:
    - Cuál es la diferencia entre entrenar con GPU y entrenar con CPU?
    - Encontrar los valores de rendimiento sobre el conjunto de test.
    - Explique los resultados de desempeño que fueron obtenidos.
    - ¿Qué pierde una imagen cuando se vectoriza?
3. Utilizar [`GridSearch`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) para buscar los hiperparámetros que ajusten mejor la red neuronal. Para eso, averiguar sobre los siguientes hiperparámetros:
    - `activation`
    - `kernel_initializer`
    - `kernel_regularizer`
    - `epochs`
4. Comparar los resultados obtenidos en el punto anterior con los obtenidos en el punto 1. Justifique.